In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

In [6]:
# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

In [7]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

In [8]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [9]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)

In [10]:
def find_closest_to_centre(b_box,centre, h,w):
    # data in b_box: x_min, Y_min, x_max, y_max, confidence/score
    conf = np.array(b_box[:,4])
    n_people = (conf>0.2).sum()
    if (n_people<=1):
        res = np.argmax(conf) #most prominent
        centroid = [((b_box[res][0]+b_box[res][2])/2)*w,((b_box[res][1]+b_box[res][3])/2)*h]
    else:
        #to find closest to center the euclidean dist. of the centroid of the bounding must be closest to centre pixel
        res = -1
        dis = 1000000
        for i in range (6):
            if (b_box[i][4]<0.2):
                continue
            c = [((b_box[i][0]+b_box[i][2])/2)*w,((b_box[i][1]+b_box[i][3])/2)*h]
            new = math.dist(centre,c)
            if (new<dis):
                dis = new
                res = i
                centroid = c
    return res,centroid
#Major assumption - subject starts off closest to centre of the frame, irrespective of who is the most prominent in the frame

In [30]:
def calculate_angle(y1,x1,y2,x2,y3,x3):

    # Calculate the angle between the three points
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    
    # Check if the angle is less than zero.
    if angle < 0:
        
        # Add 360 to the found angle.
        angle += 360
   
    # Return the calculated angle.
    return angle

In [33]:
def classify_pose(lka,rka):
    label = 'none'
    if ((lka<120 or lka > 240) or (rka<120 or rka>240)):
        label = 'sitting'
    else:
        label = 'standing'
    
    return label

In [44]:
cap = cv2.VideoCapture('D:\LifeSpark Technology\Gait Data\SP\DJI_0113.mp4')
final = []
fps = int(cap.get(cv2.CAP_PROP_FPS))
# #fourcc = cv2.VideoWriter_fourcc(*'XVID')
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
centroid = [w/2,h/2]
#video_writer = cv2.VideoWriter('KS_113_movenet.avi', cv2.VideoWriter_fourcc('P','I','M','1'), fps, (width, height), isColor=True) 
f=1
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # Resize image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 288,512)
    input_img = tf.cast(img, dtype=tf.int32)
    
    # Detection section
    results = movenet(input_img)
    b_box = results['output_0'].numpy()[:,:,51:].reshape((6,5))
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    #to find subject among others, initial frame: person nearest centre
    data = []
    key,centroid = find_closest_to_centre(b_box,centroid,h,w)   #centroid keeps changing, so same person is being tracked
    lka = calculate_angle(keypoints_with_scores[key][11][0]*h,keypoints_with_scores[key][11][1]*w,keypoints_with_scores[key][13][0]*h,keypoints_with_scores[key][13][1]*w,keypoints_with_scores[key][15][0]*h,keypoints_with_scores[key][15][1]*w)
    rka = calculate_angle(keypoints_with_scores[key][12][0]*h,keypoints_with_scores[key][12][1]*w,keypoints_with_scores[key][14][0]*h,keypoints_with_scores[key][14][1]*w,keypoints_with_scores[key][16][0]*h,keypoints_with_scores[key][16][1]*w)
    data.append(lka)
    data.append(rka)
    label = classify_pose(lka,rka)
    data.append(label)
    time = int(f/fps)
    f+=1
    data.append(time)
    final.append(data)
    # Render keypoints 
    
    keypoints_with_scores = keypoints_with_scores[key].reshape((1,17,3))  #comment this line to view all keypoints // uncomment to view only extracted keypoint
    
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.3)
    #video_writer.write(frame)
    frame = cv2.resize(frame,(960,540))
    cv2.putText(frame, label, (10, 30),cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)
    cv2.imshow('Movenet Multipose', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
#video_writer.release()
cv2.destroyAllWindows()

In [15]:
frame.shape

(1080, 1920, 3)

In [11]:
a = np.array([1,2,3,4,5])

In [13]:
a.argmax()

4

In [25]:
b_box = np.array([[0.22368601, 0.42062107, 0.7792237 , 0.51973635, 0.6374854 ],
       [0.4017529 , 0.6249735 , 0.9211679 , 0.77323395, 0.29471096],
       [0.49285173, 0.7180296 , 0.8518226 , 0.885344  , 0.27043197],
       [0.4798509 , 0.8897557 , 0.9787918 , 0.9954388 , 0.20403856],
       [0.9190839 , 0.02295218, 0.95632005, 0.03121129, 0.        ],
       [0.8638335 , 0.9166504 , 1.        , 0.9974245 , 0.00485678]])

In [30]:
conf = np.array(b_box[:,4])
conf

array([0.6374854 , 0.29471096, 0.27043197, 0.20403856, 0.        ,
       0.00485678])

In [18]:
keypoints_with_scores

array([[[0.38623562, 0.65775865, 0.32200402],
        [0.37765947, 0.661859  , 0.2423978 ],
        [0.37639356, 0.65946805, 0.37453616],
        [0.38749722, 0.6868141 , 0.40928015],
        [0.38158128, 0.67465687, 0.52859896],
        [0.44342932, 0.7245306 , 0.6959927 ],
        [0.42943874, 0.656787  , 0.559538  ],
        [0.5649513 , 0.7449355 , 0.38657108],
        [0.50232345, 0.633692  , 0.46407226],
        [0.5816127 , 0.73135847, 0.12678204],
        [0.47134003, 0.62293947, 0.14948112],
        [0.6165895 , 0.7020366 , 0.37259623],
        [0.59084797, 0.6594647 , 0.42283297],
        [0.6438366 , 0.6911451 , 0.19768706],
        [0.590214  , 0.63175434, 0.24949914],
        [0.7486596 , 0.6781819 , 0.2906392 ],
        [0.75537753, 0.67900693, 0.34597418]]], dtype=float32)

In [6]:
order = ['nose', 'left eye', 'right eye', 'left ear', 'right ear', 'left shoulder', 'right shoulder', 'left elbow', 'right elbow', 'left wrist', 'right wrist', 'left hip', 'right hip', 'left knee', 'right knee', 'left ankle', 'right ankle']
coord = ['x','y']

In [45]:
final

[[163.40251289918072, 97.29701710234302, 'sitting', 0],
 [163.26139883235768, 98.4452519689806, 'sitting', 0],
 [163.89144538332877, 97.61610387007518, 'sitting', 0],
 [164.57529837647576, 97.16110762124825, 'sitting', 0],
 [163.9930370125961, 97.13430666949102, 'sitting', 0],
 [162.56940852844298, 96.89920731865868, 'sitting', 0],
 [163.08785451187939, 97.30316292731517, 'sitting', 0],
 [163.81564379344562, 97.89857866768493, 'sitting', 0],
 [162.38240451541293, 96.73159217816773, 'sitting', 0],
 [162.25290615498386, 96.70959642426337, 'sitting', 0],
 [161.69480782691568, 95.96796095281577, 'sitting', 0],
 [161.6444300775477, 96.00969877338322, 'sitting', 0],
 [162.23707800508458, 96.22729779108423, 'sitting', 0],
 [161.74267066381395, 96.08580502969444, 'sitting', 0],
 [161.88423236984005, 95.95315193564196, 'sitting', 0],
 [162.18734167320625, 96.10033058467766, 'sitting', 0],
 [162.49653225736162, 96.07372079499942, 'sitting', 0],
 [162.6012674051601, 95.95944758394796, 'sitting', 

In [46]:
data

[161.8064113383649, 95.7764257951786, 'sitting', 1]

In [47]:
headers = ['left_knee_angle','right_knee_angle','label','timestamp']

In [ ]:
import csv
with open("filename","w+") as my_csv:
    w = csv.writer(my_csv,delimiter=',', lineterminator = '\n')
    w.writerow(headers)
    w.writerows(final)